### here i am gooing to use the set of tool like search , news search and much more 
### my example for this chainwas take the two llm and for each llm i am gooing to attach the 3 to 4 tool in that llm 

In [ ]:
#befine the llm 
from langchain_core.messages import AIMessage, HumanMessage

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0,
    max_tokens=None,
    timeout=60,
    groq_api_key="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",
    # other params...
)

llm.invoke([HumanMessage(content="hello how are you!")])


In [ ]:
# before that we want to crete the new some of the tools
import socket
import requests
def featch_ip() -> str:
    """Fetch the IP address of the meachine .
    return:
        return the ip of the meachine
    """
    try:
        ip_address = requests.get("https://api.ipify.org").text
        return ip_address
    except requests.RequestException:
        return "Unable to fetch IP address"

In [ ]:
llm_with_tools = llm.bind_tools([featch_ip])

In [ ]:
tool_call = llm_with_tools.invoke([HumanMessage(content=f"can i get the ip of my meachine", name="Lance")])

In [ ]:
tool_call

In [ ]:
tool_call.tool_calls

In [ ]:
# define the state 

from typing import Annotated
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage


class Store_message(TypedDict):
    """State to store messages."""

    messages: Annotated[list[AnyMessage], add_messages]
    name: str | None

In [ ]:
# define the node and ediges


from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
    
def greeting_node(state: Store_message):
    print("______greeting_node______")
    print("hello")
    return {"messages": state["messages"]}

def llm_with_tool_calling_node(state: Store_message):
    print("______llm_with_tool_calling_node______")
    print("calling the tool")
    # call the llm with the tool
    tool_call = llm_with_tools.invoke(state["messages"])
    # add the tool call to the state
    return {"messages": tool_call}


#add nodes

graph = StateGraph(Store_message)
graph.add_node("greeting",greeting_node)
graph.add_node("tool_calling",llm_with_tool_calling_node)

#add simple ediges

graph.add_edge(START, "greeting")
graph.add_edge("greeting","tool_calling")
graph.add_edge("tool_calling",END)
build = graph.compile()


In [ ]:
# from langsmith import Client
# client = Client(api_key="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")
# prompt = client.pull_prompt("rlm/rag-document-relevance", include_model=True)

build




In [ ]:
messages = build.invoke({"messages": HumanMessage(content="give the ip of my meachine")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
prompt

## create the chain using the tools with two models one for normal user and another one for basil user

In [ ]:
# define the state 




# ----------------------------------

In [10]:
# define the llm and llm2

from langchain_core.messages import AIMessage, HumanMessage


from langchain_groq import ChatGroq
# this llm1 is only for basil
llm1 = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    timeout=None,
    groq_api_key="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",
    # other params...
)

#this llm for everyone
llm2 = ChatGroq(
    model="qwen-qwq-32b",
    temperature=0,
    max_tokens=None,
    timeout=60,
    groq_api_key="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",
    # other params...
)

In [ ]:
llm1.invoke("hello")

In [ ]:
llm2.invoke("hello")

In [11]:
# create the bunch of tool for llm


import socket
# featch the ip address
from typing import List
import requests

#only for basil 
def featch_ip(ip:str) -> str:
    """it only featch the ip address from the meachine to get the public ip"""
    try:
        ip_address = requests.get("https://api.ipify.org").text
        return ip_address
    except requests.RequestException:
        return "Unable to fetch IP address"


#only for basil to get the what are the port are the localmeachine 

# def scanning_port(port_start:int,port_end:int)-> str:
#     """get the port details from the local meachine
#     args:
#     prot_start: staring number of port eg:4
#     prot_end: ending number of port eg:90
#     """
#     t_IP = gethostbyname("127.0.0.1")
#     values = []
#     for i in range(50, 500):
#       s = socket(port_start, port_end)
      
#       conn = s.connect_ex((t_IP, i))
#       if(conn == 0) :
#          print ('Port %d: OPEN' % (i,))
#          values+=[f"Port: OPEN' % {(i,)}"]
#       s.close()
#     return values

def scanning_port(port_start: int, port_end: int) -> List[str]:
    """
    Scans a range of ports on localhost to check if they are open.

    Args:
        port_start (int): Starting port number.
        port_end (int): Ending port number.

    Returns:
        List[str]: List of open ports with messages.
    """
    t_IP = socket.gethostbyname("127.0.0.1")
    open_ports = []

    for port in range(port_start, port_end + 1):  # range should include end
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # s.settimeout(0.5)  # Add timeout to avoid hanging
        conn = s.connect_ex((t_IP, port))
        if conn == 0:
            msg = f"Port {port}: OPEN"
            print(msg)
            open_ports.append(msg)
        s.close()

    return open_ports


#for other user function
#that only for addiciton and subraction

def addiction(a:int, b:int)-> int:
    """this function was perform the addicion and give the output in int
    args:
    a: number eg:44
    b: number eg:3
    """
    return a + b

def subraction(a:int, b:int)-> int:
    """this function was perform the subraction and give the output in int
    args:
    a: number eg:44
    b: number eg:3
    """
    return a - b


In [12]:
#binindg the along with two llms

llm1 = llm1.bind_tools([featch_ip,scanning_port])
llm2 = llm2.bind_tools([addiction,subraction])

In [ ]:
answer = llm1.invoke([HumanMessage(content="what are the port are open in my meachine from 80 to 81 port!")])
answer

In [ ]:
answer = llm1.invoke([HumanMessage(content="give my meachine ip address of 192.193.33.44")])
answer

In [ ]:
answer = llm2.invoke([HumanMessage(content="hello")])
answer

In [ ]:
answer = llm2.invoke([HumanMessage(content="hello")])
answer

In [ ]:
def tools_condition_for_llm1(state: Multi_llm_state) -> Literal["tools1", END]:
    # some logic to decide if tools1 should run
    # e.g. if a tool call is needed
    if "tool_call" in str(state["messages"][-1]):
        return "tools1"
    return END

def tools_condition_for_llm2(state: Multi_llm_state) -> Literal["tools2", END]:
    if "tool_call" in str(state["messages"][-1]):
        return "tools2"
    return END

In [13]:
from typing import Annotated
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Literal  
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# sys_msg = SystemMessage(content="You are a helpful assistant tasked with using search and performing arithmetic on a set of inputs. and also perfoem the tool exicuction ")
# sys_msg = SystemMessage(content="""
# You are a helpful assistant.

# - For general queries (such as greetings, questions about the economy, environment, sports, or casual conversation), respond directly and naturally using your own language capabilities. Do not call any tools for these types of queries. For example, if the user says "hello", you might respond with "Hello! How can I assist you today?"

# - Only use tools when the user provides a request that clearly requires computation, arithmetic, or data processing based on input values. In those cases, respond by invoking the appropriate tool.

# Your main goal is to provide a natural, conversational experience for general questions and reserve tool usage strictly for computation-related tasks.
# """)


sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

class Multi_llm_state(TypedDict):
    """State to store messages."""

    messages: Annotated[list[AnyMessage], add_messages]
    name: str | None


In [14]:
# '# define the node and condictionss


# from IPython.display import Image, display
# from langgraph.graph import StateGraph, START, END
# from langgraph.prebuilt import ToolNode
# from langgraph.prebuilt import tools_condition

# # condiiction ediges

# def routing_based_on_names(state:Multi_llm_state)-> Literal["llm1","llm2"]:
#     if state["name"] == "basil":
#         return "llm1"
#     return "llm2"


# #greeding card:

# def greeting(state:Multi_llm_state):
#     print("_____greeting_______")
#     print(f"Hello {state["name"]}")
#     return {"messages":state["messages"], "name":state["name"]}
# graph = StateGraph(Multi_llm_state)
# #define the nodes


# # node for llm1 and llm2

# def llm_1(state:Multi_llm_state):
#     print("------llm1 for basil------")
#     return {"messages": llm1.invoke(state["messages"])}

# def llm_2(state:Multi_llm_state):
#     print("------llm2 for other member------")
#     return {"messages": llm2.invoke(state["messages"])}


# #define the nodes
# graph.add_node("greeting",greeting)
# graph.add_node("llm1",llm_1)
# graph.add_node("llm2",llm_2)
# graph.add_node("tools1",ToolNode([featch_ip,scanning_port]))
# graph.add_node("tools2",ToolNode([addiction,subraction]))


# #add the ediges and condiction ediges

# graph.add_edge(START,"greeting")
# graph.add_conditional_edges("greeting",routing_based_on_names)
# graph.add_conditional_edges("llm1",tools_condition_for_llm1)
# graph.add_conditional_edges("llm2",tools_condition_for_llm2)
# # graph.add_edge("tools1", END)
# # graph.add_edge("tools2", END)

# build = graph.compile()


# define the node and condictionss


from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

# condiiction ediges

def routing_based_on_names(state:Multi_llm_state)-> Literal["llm1","llm2"]:
    if state["name"] == "basil":
        return "llm1"
    return "llm2"


#greeding card:

def greeting(state:Multi_llm_state):
    print("_____greeting_______")
    print(f"Hello {state["name"]}")
    return {"messages":state["messages"], "name":state["name"]}
graph = StateGraph(Multi_llm_state)
#define the nodes


# node for llm1 and llm2

def llm_1(state:Multi_llm_state):
    print("------llm1 for basil------")
    return {"messages": [llm1.invoke([sys_msg] + state["messages"])]}

def llm_2(state:Multi_llm_state):
    print("------llm2 for other member------")
    return {"messages": llm2.invoke([sys_msg] + state["messages"])}


#define the nodes
graph.add_node("greeting",greeting)
graph.add_node("llm1",llm_1)
graph.add_node("llm2",llm_2)
# graph.add_node("tools",ToolNode([featch_ip,scanning_port]))
graph.add_node("tools",ToolNode([addiction,subraction]))


#add the ediges and condiction ediges

graph.add_edge(START,"greeting")
graph.add_conditional_edges("greeting",routing_based_on_names)
# graph.add_conditional_edges("llm1",tools_condition)
graph.add_conditional_edges("llm2",tools_condition)
graph.add_edge("tools", "llm2")
# graph.add_edge("tools2", "llm2")

build = graph.compile()


In [ ]:
build

In [6]:
messages = build.invoke({"messages": [sys_msg, HumanMessage(content="give me an port scane for my meachine and port of 80 to port of 81")],"name":"basil"})
for m in messages['messages']:
    
    m.pretty_print()

_____greeting_______
Hello basil
------llm1 for basil------
================================ System Message ================================

You are a helpful assistant tasked with performing arithmetic on a set of inputs.
================================ Human Message =================================

give me an port scane for my meachine and port of 80 to port of 81
================================== Ai Message ==================================
Tool Calls:
  scanning_port (call_rxea)
 Call ID: call_rxea
  Args:
    port_start: 80
    port_end: 81


In [17]:
messages = build.invoke({"messages": HumanMessage(content="what was the addiciton of 33 and 33"),"name":"suresh"})
for m in messages['messages']:
    m.pretty_print()

_____greeting_______
Hello suresh
------llm2 for other member------
------llm2 for other member------
================================ Human Message =================================

what was the addiciton of 33 and 33
================================== Ai Message ==================================
Tool Calls:
  addiction (call_ng6m)
 Call ID: call_ng6m
  Args:
    a: 33
    b: 33
================================= Tool Message =================================
Name: addiction

66
================================== Ai Message ==================================

The addition of 33 and 33 is **66**.


In [ ]:
messages = build.invoke({"messages": HumanMessage(content="hello how are you"),"name":"dhanesh"})
for m in messages['messages']:
    m.pretty_print()